### Hndling Imbalanced Datasets

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import mlflow

# Setup MLFlow Tracking Server
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
mlflow.set_tracking_uri(mlflow_tracking_uri)

e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\mlflow\protos\service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [3]:
# Set or Create an experiment
mlflow.set_experiment("Exr 4- Handling Imbalanced Data")

<Experiment: artifact_location='mlflow-artifacts:/bd78764ab56b4fa4b20d1a901a8de44e', creation_time=1766739441185, experiment_id='5', last_update_time=1766739441185, lifecycle_stage='active', name='Exr 4- Handling Imbalanced Data', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [4]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [5]:
df = pd.read_csv('./artifacts/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
print(df.shape)
df.sample(5)

(36662, 2)


,clean_comment,category
10673,appeasement didi not anything votebank rahul k...,1
31984,lmao hysterical,-1
15675,becoming like chinese dictatorship,1
24401,people section indeed affected rich lost money...,1
5494,pretty much clear people india definitely not ...,1


In [6]:
import os

ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

### Techiniques Used are Class_Weights, OverSampling, ADASYN, UnderSampling, SMOTE_ENN

In [7]:
# Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3)    # Trigram setting
    max_features = 1000

    # Split the data into training and test datasets
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42)

    # Vectorization using TF-IDF, fit on training data only
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train_vec = vectorizer.fit_transform(X_train)     # Fit on training data
    X_test_vec = vectorizer.transform(X_test)       # Transform test data

    # Handle class imbalance based on the selected method (only applied to the training set)
    if imbalance_method == 'class_weights':
        # Use class_weight in Random Forest
        class_weight = 'balanced'
    else:
        class_weight = 'None'   # Do not apply class_weight if using resampling

        # Resampling Techniques (only applied to the training dataset)
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

    # Define and train a RandomForest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a descripion
        mlflow.set_tag("description", f"Random Forest with TF-IDF Trigrams, imbalance handling method= {imbalance_method}")

         # Log Vectorizer parameters
        mlflow.log_param("vectorizer_type: ", "TF-IDF")
        mlflow.log_param("ngram_range: ", ngram_range)
        mlflow.log_param("vectorizer_max_features: ", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train_vec, y_train)

        # Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log Accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("Accuracy: ", accuracy)

        # Log Classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    if metric != "support":
                        mlflow.log_metric(f"{label}_{metric}", value)

        # Log Confusion Matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")

        # Save and log the confusion matrix plot
        filename = f"Exp4_TF_IDF_Trigrams_{imbalance_method}_ConfusionMatrix.png"
        cm_path = os.path.join(ARTIFACT_DIR, filename)
        plt.savefig(cm_path)
        mlflow.log_artifact(cm_path)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"Random_Forest_Model_TF_IDF_Trigrams_imbalance_{imbalance_method}")


In [8]:
# Run experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)

2025/12/26 17:19:23 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2025/12/26 17:21:33 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register conc